In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import glob

import matplotlib.pyplot as plt
import seaborn as sns

import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
from tensorflow import keras
import tensorflow as tf
import keras.backend as K
from tensorflow.keras import optimizers, callbacks, layers, losses
from tensorflow.keras.layers import Dense, Concatenate, Activation, Add, BatchNormalization, Dropout, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Input, GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential, load_model

import random

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

def mape(y_true, y_pred):
    return 100*np.mean((np.abs(y_true-y_pred)/y_true))

def mish(x):
    return x*K.tanh(K.softplus(x))

def decay(epochs):
    init = 1e-3
    drop = 25
    ratio = 0.9
    return max(5e-5, (init * (ratio ** (epochs//drop))))    

es = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
lrs = callbacks.LearningRateScheduler(decay, verbose=0)


Using TensorFlow backend.


In [3]:
# TODO
## instance normalization instead of batchnormalization
def ConvBlock(fs, ks, strides=1, activation=None, padding='same', bn=True):
    def f(x):
        x = Conv2D(fs, ks, strides, padding=padding)(x)
        if bn:
            x = BatchNormalization()(x)
        if activation:
            x = Activation(activation)(x)
        return x
    return f

class ReflectionPadding2D(layers.Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        padding_width, padding_height = self.padding
        padding_tensor = [
            [0, 0],
            [padding_height, padding_height],
            [padding_width, padding_width],
            [0, 0],
        ]
        return tf.pad(input_tensor, padding_tensor, mode="REFLECT")
    

def ResBlock(ks, strides=1, n_conv=2, activation=None, padding='valid', bn=True):
    def f(x):
        dim = x.shape[-1]
        short_cut = x
        
        for _ in range(n_conv):
            x = ReflectionPadding2D()(x)
            x = ConvBlock(dim, ks, strides, activation, padding, bn)(x)
        
        x = Add()([short_cut, x])
        return x
        
    return f

In [4]:
def build_resnet(shape):
    n_down = 2
    ds = 32

    n_res = 4
    rs = 64
    
    inputs = Input(shape=shape)
    
    x = ConvBlock(ds, 3, 2, activation=mish)(inputs)
    
    for i in range(1, n_down):
        x = ConvBlock(ds*2**(i+1), 3, 2, activation=mish)(x)
        
    for i in range(n_res):
        x = ResBlock(3, activation=mish)(x)
    
    x = GlobalMaxPooling2D()(x)
    outputs = Dense(10, activation='softmax')(x)
    return Model(inputs, outputs)

In [5]:
m = build_resnet((120, 120, 3))

In [6]:
m.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 120, 120, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 60, 60, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 60, 60, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 60, 60, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________